In [1]:
from gurobipy import *
# from math import *
# import math
import numpy as np
import pandas as pd
import timeit
from xlwt import Workbook
from math import floor
import openpyxl
import plotly.graph_objs as go
import plotly.offline as pyo

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Input data

In [2]:
ac_types = 3 # Number of aircraft types
LF = 0.8 # Load factor
BT = 10 * 7 # Block time [h/week]
f = 1.42 # Fuel cost [USD/gallon]

In [3]:
data_demand = pd.read_csv('Group_6_Demand.csv')
demand = np.array(data_demand)[::,1::]  # Demand matrix between airports i and j

In [4]:
airports_data = pd.read_csv('Group_6_Airport_info.csv')
Airports = airports_data['ICAO Code'].tolist()
airports = range(len(Airports))
runway_lengths = airports_data['Runway (m)'].tolist()

In [5]:
# Matrix with max runway length between origin and destination
runways = np.zeros((len(runway_lengths), len(runway_lengths)))
for origin in range(len(runway_lengths)):
    for destination in range(len(runway_lengths)):
        if origin != destination:
            if runway_lengths[origin] < runway_lengths[destination]:
                runways[origin][destination] = runway_lengths[origin]
            else:
                runways[origin][destination] = runway_lengths[destination]

In [6]:
data_distances = pd.read_csv('Group_6_Distances.csv')
distances = np.array(data_distances)[::,1::]  

In [7]:
# Aircraft parameters
ac_data = pd.read_csv('Aircraft_info.csv')

speed = ac_data['Speed'].tolist()
seats = ac_data['Seats'].tolist()
TAT = ac_data['TAT'].tolist()
ac_range = ac_data['Range'].tolist()
runway = ac_data['Runway'].tolist()
lease = ac_data['Lease_c'].tolist()
op_cost = ac_data['Operating_c'].tolist()
time_cost_param = ac_data['Time_c'].tolist()
fuel_cost_param = ac_data['Fuel_c'].tolist()

keys = ['speed', 'seats', 'TAT', 'ac_range', 'runway', 'lease', 'op_cost', 'time_cost_param', 'fuel_cost_param']
values = [speed, seats, TAT, ac_range, runway, lease, op_cost, time_cost_param, fuel_cost_param]
Aircraft = dict(zip(keys, values))
print(Aircraft)

{'speed': [820, 850, 870], 'seats': [70, 150, 320], 'TAT': [35, 45, 60], 'ac_range': [3300, 6300, 12000], 'runway': [1600, 1800, 2600], 'lease': [34000, 70000, 180000], 'op_cost': [600, 1250, 2000], 'time_cost_param': [775, 1400, 2800], 'fuel_cost_param': [2.0, 3.75, 7.0]}


In [8]:
# 'TAT' list with values converted to hours
TAT_hours = [tat / 60 for tat in Aircraft['TAT']]
Aircraft['TAT'] = TAT_hours

# Yield between airports i and j - Revenue per RPK flown
Yield = np.zeros((len(Airports),len(Airports)))
for i in range(len(distances)):
    for j in range(len(distances)):
        if i != j:
            Yield[i][j] = 5.9 * distances[i][j] ** (-0.76) + 0.043

In [9]:
# Range matrix
a = {}
for k in range(ac_types):
    for i in range(len(distances)):
        for j in range(len(distances[i])):
            a[i,j,k] = 0
            if distances[i][j] <= Aircraft['ac_range'][k] :
                a[i,j,k] = 10000


In [10]:
# Runway matrix
run = {}
for k in range(ac_types):
    for i in range(len(runways)):
        for j in range(len(runways[i])):
            run[i,j,k] = 0
            if runways[i][j] >= Aircraft['runway'][k] :
                run[i,j,k] = 10000

## Optimziation model.

In [11]:
m = Model('model')

# Decision variables
x = {} # direct pax flow from airport i to airport j
w = {} # pax flow from airport i to airport j with transfer at hub
z = {} # number of flights (frequency) from airport i to airport j
AC = {} # number of aircraft of type k

# Setting the hub to Milan (LIMC)
g = np.ones(34) 
g[0] = 0   

# Hub and spokes dictionaris 
hub = {}
spokes = {}
for j in range(len(demand)):
    if g[j] == 0: 
        hub[j] = j
    else:
        spokes[j] = j


# Objective function 
for i in airports:
    for j in airports:
        x[i,j] = m.addVar(obj = Yield[i][j] * distances[i][j], lb=0, vtype=GRB.INTEGER)
        w[i,j] = m.addVar(obj = 0.9 * Yield[i][j] * distances[i][j], lb=0, vtype=GRB.INTEGER) # 10% lower revenue in case of transfer at hub

        for k in range(ac_types):
            C_O_ac = Aircraft['op_cost'][k]
            C_T_ac = Aircraft['time_cost_param'][k] * distances[i][j] / Aircraft['speed'][k]
            C_F_ac = Aircraft['fuel_cost_param'][k] * f * distances[i][j] / 1.5

            if g[i] == 0 or g[j] == 0: # if origin/destination is the hub
                z[i,j,k] = m.addVar(obj= -0.7 * (C_O_ac + C_T_ac + C_F_ac), lb=0, vtype=GRB.INTEGER) # 30% lower operating costs
            else:
                z[i,j,k] = m.addVar(obj= - (C_O_ac + C_T_ac + C_F_ac), lb=0, vtype=GRB.INTEGER)

for k in range(ac_types):
    AC[k] =  m.addVar(obj= - (Aircraft['lease'][k]), lb=0, vtype=GRB.INTEGER)

m.update()
m.setObjective(m.getObjective(), GRB.MAXIMIZE)


# Constraints
for i in airports:
    for j in airports:
        # C1: Demand constraint
        m.addConstr(x[i,j] + w[i,j] <= demand[i][j])

        # C1*: Transfer pax constraint (also demand-related)
        m.addConstr(w[i,j] <= demand[i][j] * g[i] * g[j])

        # C2: Capacity constraint
        m.addConstr(x[i,j] + quicksum(w[i,m] for m in range(len(demand))) * (1 - g[j]) + quicksum(w[m,j] for m in range(len(demand))) 
                    * (1 - g[i])<= quicksum(z[i,j,k] * floor(Aircraft['seats'][k] * LF) for k in range(ac_types)))

        # C5: Range constraint
        for k in range(ac_types):
            m.addConstr(z[i,j,k] <= a[i,j,k])

        # "C6": Runway length constraint (new!)
        for k in range(ac_types):
            m.addConstr(z[i,j,k] <= run[i,j,k])

    # C3: In-out constraint
    for k in range(ac_types):
        m.addConstr(quicksum(z[i,j,k] for j in range(len(demand[i]))) == quicksum(z[j,i,k] for j in range(len(demand[i]))))

# C4: Time constraint
for k in range(ac_types):
    m.addConstr(quicksum(quicksum((distances[i][j] / Aircraft['speed'][k] + 1.5 * Aircraft['TAT'][k]) * z[i,j,k] for j in hub) for i in range(len(demand)))
    + quicksum(quicksum((distances[i][j] / Aircraft['speed'][k] + Aircraft['TAT'][k]) * z[i,j,k] for j in spokes) for i in range(len(demand))) <= BT * AC[k])

m.update()

m.setParam('OutputFlag', 1)

m.optimize()
m.write("testout.sol")
status = m.status


Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-04
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9903 rows, 5448 columns and 25940 nonzeros
Model fingerprint: 0xf6798c41
Variable types: 0 continuous, 5448 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 3e+02]
  Objective range  [2e+01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 8062 rows and 1832 columns
Presolve time: 0.02s
Presolved: 1841 rows, 3616 columns, 11494 nonzeros
Variable types: 0 continuous, 3616 integer (1 binary)

Root relaxation: objective 3.089223e+05, 2159 iterations, 0.05 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

## Printing solutions.

In [ ]:
for i in airports:
    for j in airports:
        print(Airports[i], ' to ', Airports[j], x[i,j].X)

In [ ]:
for i in airports:
    for j in airports:
        print(Airports[i], ' to ', Airports[j], w[i,j].X)

In [55]:
for i in airports:
    for j in airports:
        for k in range(num_ac_types):
            if z[i,j,k].X >0:
                print(Airports[i], ' to ', Airports[j], z[i,j,k].X)


Frequencies:----------------------------------

LIMC  to  EDDF 1.0
LIMC  to  EDDF 4.0
LIMC  to  RJAA 1.0
LIMC  to  KJFK 1.0
LIMC  to  SBGL 3.0
LIMC  to  LEMD 3.0
LIMC  to  LTFM 1.0
LIMC  to  SAEZ 5.0
LIMC  to  ZSPD 2.0
LIMC  to  EDDM 12.0
LIMC  to  EDDM 2.0
LIMC  to  LFPG 6.0
LIMC  to  VIDP 2.0
LIMC  to  OMDB 2.0
LIMC  to  EDDT 1.0
LIMC  to  EDDT 2.0
LIMC  to  MMMX 3.0
LIMC  to  EFHK 1.0
LIMC  to  EBBR 1.0
LIMC  to  EGPH 3.0
LIMC  to  LKPR 2.0
LIMC  to  FACT 5.0
LIMC  to  LGAV 4.0
LIMC  to  BIKF 3.0
LIMC  to  ESSA 2.0
LIMC  to  EIDW 1.0
LIMC  to  LPPT 4.0
LIMC  to  EETN 2.0
LIMC  to  LFQQ 1.0
LIMC  to  EFRO 1.0
LIMC  to  LDZA 3.0
LIMC  to  EHEH 2.0
EDDF  to  LIMC 4.0
EDDF  to  EDDM 1.0
EDDF  to  EDDM 1.0
EDDF  to  LFPG 2.0
EDDF  to  EDDT 1.0
EDDF  to  EGPH 4.0
EDDF  to  LKPR 1.0
EDDF  to  LFQQ 1.0
EDDF  to  LDZA 1.0
EDDF  to  EHEH 1.0
RJAA  to  LIMC 1.0
KJFK  to  LIMC 1.0
SBGL  to  LIMC 3.0
LEMD  to  LIMC 3.0
LEMD  to  LPPT 1.0
LTFM  to  LIMC 1.0
SAEZ  to  LIMC 5.0
ZSPD  to  LIMC 2.0


## Creating Excel with the results.

In [29]:
# Create an Excel workbook
wb = openpyxl.Workbook()

default_sheet = wb.sheetnames[0]
wb.remove(wb[default_sheet])

# Create a sheet for revenue and cargo delivered

final_values = wb.create_sheet(title="Results")
direct_flights = wb.create_sheet(title="Direct flights")
hub_flights = wb.create_sheet(title="Transfer flights at hub")
aircraft1 = wb.create_sheet(title="Aircraft 1")
aircraft2 = wb.create_sheet(title="Aircraft 2")
aircraft3 = wb.create_sheet(title="Aircraft 3")


final_values.append(["Contribution [EUR/week] :", m.objVal])
final_values.append(["Gap [%] ", m.MIPgap * 100])
for k in range(ac_types):
    final_values.append(['Aircraft %s:' % str(k + 1), AC[k].x])


for i in range(len(Airports)):
    
    direct_flights.cell(row=i + 2, column=1, value=Airports[i])
    direct_flights.cell(row=1, column=i + 2, value=Airports[i])

    hub_flights.cell(row=i + 2, column=1, value=Airports[i])
    hub_flights.cell(row=1, column=i + 2, value=Airports[i])

    aircraft1.cell(row=i + 2, column=1, value=Airports[i])
    aircraft1.cell(row=1, column=i + 2, value=Airports[i])

    aircraft2.cell(row=i + 2, column=1, value=Airports[i])
    aircraft2.cell(row=1, column=i + 2, value=Airports[i])

    aircraft3.cell(row=i + 2, column=1, value=Airports[i])
    aircraft3.cell(row=1, column=i + 2, value=Airports[i])

    
    for j in range(len(Airports)):
        direct_flights.cell(row=1, column=j + 2, value=Airports[j])
        hub_flights.cell(row=1, column=j + 2, value=Airports[j])
       
        
        direct_flights.cell(row=i + 2, column=j + 2, value=x[i, j].x)
        hub_flights.cell(row=i + 2, column=j + 2, value=w[i, j].x)        
        

        for k in range(ac_types):
            aircraft1.cell(row=i + 2, column=j + 2, value=z[i,j,0].x)
            aircraft2.cell(row=i + 2, column=j + 2, value=z[i,j,1].x)
            aircraft3.cell(row=i + 2, column=j + 2, value=z[i,j,2].x)

wb.save("Assignmnet1_Problem1_results.xlsx")

print("Results saved to Excel.")

Results saved to Excel.


In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo

optimized_cargo_amounts = [x[i][j].X for i in range(airports) for j in range(airports)]


airport_name = airport_data['ICAO Code']
hub = 'AUH'  # Change this to your desired hub airport code



# Create a Figure and add Scattergeo for airports and lines from the hub
fig = go.Figure()

# Add Scattergeo for airports
fig.add_trace(go.Scattergeo(
    lat=airport_data['Latitude'],
    lon=airport_data['Longitude'],
    text=airport_name,
    mode='markers+text',
    marker=dict(color='blue', size=8),
    textposition='bottom center',
    textfont=dict(size=15)
))



        
# Update layout 
fig.update_layout(
    geo=dict(
        resolution=50,
        showland=True,
        showcountries=True,
        showocean=True,
        countrycolor='black',
        landcolor='white',
        oceancolor='lightblue',
        projection_type='equirectangular'
    ),
    width=1920,  # Set width to screen width
    height=1080,  # Set height to screen height
)
    

# Finally, display the map
pyo.plot(fig, filename='map_with_cargo_big.html', auto_open=True)


In [ ]:

# Create a dictionary to map route indices to colors
route_color_dict = {}
for idx, flight_legs in enumerate(flight_sequence):
    route = tuple(tuple(leg) for leg in flight_legs)
    route_color_dict[route] = f'#{random.randint(0, 0xFFFFFF):06x}' 

# Iterate through each flight sequence to add lines on the map with cargo information
for seq_idx, flight_seq in enumerate(flight_sequence):
    route = tuple(tuple(leg) for leg in flight_seq)
    color = route_color_dict.get(route, 'black')  # Default to black for unknown routes

    for leg_idx, flight_leg in enumerate(flight_seq):
        start_airport, end_airport = flight_leg

        cargo = optimized_cargo_amounts[seq_idx]
        if cargo > 0:
            start_lat = airport_data[airport_data['IATA'] == start_airport]['Latitude'].iloc[0]
            start_lon = airport_data[airport_data['IATA'] == start_airport]['Longitude'].iloc[0]
            end_lat = airport_data[airport_data['IATA'] == end_airport]['Latitude'].iloc[0]
            end_lon = airport_data[airport_data['IATA'] == end_airport]['Longitude'].iloc[0]

            fig.add_trace(go.Scattergeo(
                lat=[start_lat, end_lat],
                lon=[start_lon, end_lon],
                mode='lines',
                line=dict(color=color, width=2),
                hoverinfo='text',
                text=f'Route: {flight_seq}<br>Cargo: {cargo}'  # Display cargo info on hover
          
        ))

## Mapping 

In [12]:
direct_flow = [[x[i, j].X for j in range(len(airports))] for i in range(len(airports))]
indirect_flow = [[w[i, j].X for j in range(len(airports))] for i in range(len(airports))]

ac1_freq =[[z[i, j, 0].X for j in range(len(airports))] for i in range(len(airports)) ]
ac2_freq =[[z[i, j, 1].X for j in range(len(airports))] for i in range(len(airports)) ]
ac3_freq =[[z[i, j, 2].X for j in range(len(airports))] for i in range(len(airports)) ]



# List of airport codes
airport_codes = airports_data['ICAO Code']
airport_name = airports_data['ICAO Code']

## Direct flow mapping

In [15]:

data1 = []

# Loop through the dirext_flow to create lines for the direct flow
for i in range(len(airports)):
    for j in range(len(airports)):
        data1.append(
            go.Scattergeo(
                lon=[airports_data['Longitude (deg)'][i], airports_data['Longitude (deg)'][j]],
                lat=[airports_data['Latitude (deg)'][i], airports_data['Latitude (deg)'][j]],
                mode='lines',
                line=dict(width=direct_flow[i][j] / 200, color='red'),  # Adjust line width based on cargo amount
                # opacity=0.4,
                hoverinfo='text',
                text=f'From: {airport_codes[i]}<br>To: {airport_codes[j]}<br>Cargo Amount: {direct_flow[i][j]}'
            )
        )

# Create Scattergeo for airports
airport_trace = go.Scattergeo(
    lat=airports_data['Latitude (deg)'],
    lon=airports_data['Longitude (deg)'],
    text=airport_codes,
    mode='markers+text',
    marker=dict(color='blue', size=8),
    textposition='bottom center',
    textfont=dict(size=15)
)

data1.append(airport_trace)

# Create layout for the map
layout1 = go.Layout(
    title='Direct Passenger Flow between Airports',
    showlegend=False,
    geo=dict(
        resolution=50,
        showland=True,
        showcountries=True,
        showocean=True,
        countrycolor='black',
        landcolor='white',
        oceancolor='lightblue',
        projection_type='equirectangular'
    ),
    width=1920,  # Screen width
    height=1080,  # Screen height
)


fig = go.Figure(data=data1, layout=layout1)


pyo.plot(fig, filename='map_with_direct_passenger_flow.html', auto_open=True)


'map_with_direct_passenger_flow.html'

## Transfer at the hub flow mapping

In [67]:

data2 = []

# Loop through the dirext_flow to create lines for the direct flow
for i in range(len(airports)):
    for j in range(len(airports)):
        data2.append(
            go.Scattergeo(
                lon=[airports_data['Longitude (deg)'][i], airports_data['Longitude (deg)'][j]],
                lat=[airports_data['Latitude (deg)'][i], airports_data['Latitude (deg)'][j]],
                mode='lines',
                line=dict(width=indirect_flow[i][j] / 200, color='blue'),  # Adjust line width based on cargo amount
                # opacity=0.4,
                hoverinfo='text',
                text=f'From: {airport_codes[i]}<br>To: {airport_codes[j]}<br>Cargo Amount: {indirect_flow[i][j]}'
            )
        )

# Create Scattergeo for airports
airport_trace = go.Scattergeo(
    lat=airports_data['Latitude (deg)'],
    lon=airports_data['Longitude (deg)'],
    text=airport_codes,
    mode='markers+text',
    marker=dict(color='blue', size=8),
    textposition='bottom center',
    textfont=dict(size=15)
)

data2.append(airport_trace)

# Create layout for the map
layout2 = go.Layout(
    title='Passenger Flow between Airports with change at the hub',
    showlegend=False,
    geo=dict(
        resolution=50,
        showland=True,
        showcountries=True,
        showocean=True,
        countrycolor='black',
        landcolor='white',
        oceancolor='lightblue',
        projection_type='equirectangular'
    ),
    width=1920,  # Screen width
    height=1080,  # Screen height
)


fig = go.Figure(data=data2, layout=layout2)


pyo.plot(fig, filename='map_with_indirect_passenger_flow.html', auto_open=True)


'map_with_indirect_passenger_flow.html'

### Aircraft 1 network 

In [16]:

data3 = []

# Loop through the dirext_flow to create lines for the direct flow
for i in range(len(airports)):
    for j in range(len(airports)):
        for k in range(ac_types):
            data3.append(
                go.Scattergeo(
                    lon=[airports_data['Longitude (deg)'][i], airports_data['Longitude (deg)'][j]],
                    lat=[airports_data['Latitude (deg)'][i], airports_data['Latitude (deg)'][j]],
                    mode='lines',
                    line=dict(width=ac1_freq[i][j] / 200, color='green'),  
                    hoverinfo='text',
                    text=f'From: {airport_codes[i]}<br>To: {airport_codes[j]}<br>Cargo Amount: {ac1_freq[i][j]}'
            )
        )

# Create Scattergeo for airports
airport_trace = go.Scattergeo(
    lat=airports_data['Latitude (deg)'],
    lon=airports_data['Longitude (deg)'],
    text=airport_codes,
    mode='markers+text',
    marker=dict(color='blue', size=8),
    textposition='bottom center',
    textfont=dict(size=15)
)

data3.append(airport_trace)

# Create layout for the map
layout3 = go.Layout(
    title='Passenger Flow between Airports with change at the hub',
    showlegend=False,
    geo=dict(
        resolution=50,
        showland=True,
        showcountries=True,
        showocean=True,
        countrycolor='black',
        landcolor='white',
        oceancolor='lightblue',
        projection_type='equirectangular'
    ),
    width=1920,  # Screen width
    height=1080,  # Screen height
)


fig = go.Figure(data=data3, layout=layout3)


pyo.plot(fig, filename='map_with_ac1_passenger_flow.html', auto_open=True)


'map_with_ac1_passenger_flow.html'

### Aircraft 2 netowrk

In [17]:

data4 = []

# Loop through the passenger flow to create lines for each aircraft type
for k in range(ac_types): 
    for i in range(len(airports)):
        for j in range(len(airports)):
            data4.append(
                go.Scattergeo(
                    lon=[airports_data['Longitude (deg)'][i], airports_data['Longitude (deg)'][j]],
                    lat=[airports_data['Latitude (deg)'][i], airports_data['Latitude (deg)'][j]],
                    mode='lines',
                    line=dict(width=ac2_freq[i][j] / 200, color='orange'),  
                    hoverinfo='text',
                    text=f'From: {airport_codes[i]}<br>To: {airport_codes[j]}<br>Passenger Flow: {ac2_freq[i][j]}'
                )
            )

# Create Scattergeo for airports
airport_trace = go.Scattergeo(
    lat=airports_data['Latitude (deg)'],
    lon=airports_data['Longitude (deg)'],
    text=airport_codes,
    mode='markers+text',
    marker=dict(color='blue', size=8),
    textposition='bottom center',
    textfont=dict(size=15)
)

data4.append(airport_trace)

# Create layout for the map
layout4 = go.Layout(
    title='Aicraft 2 network',
    showlegend=False,
    geo=dict(
        resolution=50,
        showland=True,
        showcountries=True,
        showocean=True,
        countrycolor='black',
        landcolor='white',
        oceancolor='lightblue',
        projection_type='equirectangular',
        center=dict(lon=airports_data[airports_data['ICAO Code'] == 'LIMC']['Longitude (deg)'].values[0],
                    lat=airports_data[airports_data['ICAO Code'] == 'LIMC']['Latitude (deg)'].values[0]),
    ),
    width=1920,  # Screen width
    height=1080,  # Screen height
)

fig = go.Figure(data=data4, layout=layout4)

pyo.plot(fig, filename='map_with_ac2_passenger_flow.html', auto_open=True)


'map_with_ac2_passenger_flow.html'

### Aircraft 3 network 

In [18]:

data5 = []

# Loop through the passenger flow to create lines for each aircraft type
for k in range(ac_types): 
    for i in range(len(airports)):
        for j in range(len(airports)):
            data5.append(
                go.Scattergeo(
                    lon=[airports_data['Longitude (deg)'][i], airports_data['Longitude (deg)'][j]],
                    lat=[airports_data['Latitude (deg)'][i], airports_data['Latitude (deg)'][j]],
                    mode='lines',
                    line=dict(width=ac3_freq[i][j] / 200, color='black'),  # Adjust line width based on passenger flow
                    hoverinfo='text',
                    text=f'From: {airport_codes[i]}<br>To: {airport_codes[j]}<br>Passenger Flow: {ac3_freq[i][j]}'
                )
            )

# Create Scattergeo for airports
airport_trace = go.Scattergeo(
    lat=airports_data['Latitude (deg)'],
    lon=airports_data['Longitude (deg)'],
    text=airport_codes,
    mode='markers+text',
    marker=dict(color='blue', size=8),
    textposition='bottom center',
    textfont=dict(size=15)
)

data5.append(airport_trace)

# Create layout for the map
layout5 = go.Layout(
    title='Aicraft 2 network',
    showlegend=False,
    geo=dict(
        resolution=50,
        showland=True,
        showcountries=True,
        showocean=True,
        countrycolor='black',
        landcolor='white',
        oceancolor='lightblue',
        projection_type='equirectangular',
        center=dict(lon=airports_data[airports_data['ICAO Code'] == 'LIMC']['Longitude (deg)'].values[0],
                    lat=airports_data[airports_data['ICAO Code'] == 'LIMC']['Latitude (deg)'].values[0]),
    ),
    width=1920,  # Screen width
    height=1080,  # Screen height
)

fig = go.Figure(data=data5, layout=layout5)

pyo.plot(fig, filename='map_with_ac3_passenger_flow.html', auto_open=True)


'map_with_ac3_passenger_flow.html'